# Synthetic Dataset Verification
* Heteroscedastic standard deviation implementation.
* Full Bayesian regression

## Residuals Partition
**Goal:**<br>
Partition total residuals into within-event-site, between-event, between-event-path, and site residuals through Bayesian regression.

### Set Up

In [ ]:
#load libraries
#general
import os
import sys
import pathlib
#string libraries
import re
#arithmetic libraries
import numpy as np
#statistics libraries
import pandas as pd
#plot libraries
import matplotlib as mpl
from matplotlib import pyplot as plt
import arviz as az
#widgets
import ipywidgets
#stan library
import cmdstanpy
#user functions


### Define Variables

In [ ]:
#filename for aleatory standard deviations
fname_df_hyp    = "../../../Data/gmm_ergodic/syndata/synds_test3_gmm_coefficients.csv"
#filename total residuals dataframe
fname_df_totres = "../../../Data/gmm_ergodic/syndata/synds_test3_tot_residuals.csv"

#regression name
name_gmm = "gmm_ss_wo_mag_req"
#total residual column name
cname_dT = name_gmm + '_dT'
#output filename
fname_out = name_gmm

#regression info
fname_stan_model = '../../stan_lib/partition_residuals_eq_path_unconstrained_v0.stan'
#iteration samples
# n_iter_warmup   = 500
# n_iter_sampling = 500
n_iter_warmup   = 1000
n_iter_sampling = 1000
# n_iter_warmup   = 10000
# n_iter_sampling = 10000
#MCMC parameters
n_chains        = 6
adapt_delta     = 0.8
max_treedepth   = 10

#output directory
dir_out = "../../../Data/gmm_ergodic/syndata_unconstr/"
dir_fig = dir_out + 'figures/'

### Load Data

In [ ]:
#hyper-parameters
df_coeffs = pd.read_csv(fname_df_hyp, index_col=0)
#total residuals
df_totres = pd.read_csv(fname_df_totres)

### Set Up Variables

In [ ]:
#aleatory standard deviations
phi0 = df_coeffs.loc[name_gmm,'phi']
phiS = df_coeffs.loc[name_gmm,'phiS']
tau0 = df_coeffs.loc[name_gmm,'tau']
tauP = df_coeffs.loc[name_gmm,'tauP']

#number of data points
n_data = len(df_totres)

#earthquake data
data_eq_all = df_totres[['event_id','mag','scl_dB','scl_dBP']].values
_, eq_idx, eq_inv, eq_cnt = np.unique(df_totres[['event_id']], axis=0, return_inverse=True, return_index=True, return_counts=True)
data_eq = data_eq_all[eq_idx,:]
#X_eq = data_eq[:,[3,4]] #earthquake coordinates
#create earthquake ids for all records (1 to n_eq)
eq_id = eq_inv + 1
n_eq = len(data_eq)

#station data
data_sta_all = df_totres[['station_id','vs30','scl_dS']].values
_, sta_idx, sta_inv, sta_cnt = np.unique( df_totres[['station_id']].values, axis = 0, return_inverse=True, return_index=True, return_counts=True)
data_sta = data_sta_all[sta_idx,:]
#X_sta = data_sta[:,[2,3]] #station coordinates
#create station indices for all records (1 to n_sta)
sta_id = sta_inv + 1
n_sta = len(data_sta)

#within-event residual scaling
scl_dWS = df_totres['scl_dWS'].values

#rupture distance
r_rup = df_totres['rrup'].values

#total residuals
deltaT = df_totres[cname_dT].to_numpy().copy()

### Regression

In [ ]:
#create output directory
pathlib.Path(dir_out).mkdir(parents=True, exist_ok=True) 
pathlib.Path(dir_fig).mkdir(parents=True, exist_ok=True) 

stan_data = {'N':        n_data,
             'NEQ':      n_eq,
             'NSTA':     n_sta,
             'eq':       eq_id,                  #earthquake id
             'sta':      sta_id,                 #station id
             'phi0':     phi0,
             'phiS':     phiS,
             'tau0':     tau0,
             'tauP':     tauP,
             'sclWS':    scl_dWS,
             'sclS':     data_sta[:,2],
             'sclB':     data_eq[:,2],
             'sclBP':    data_eq[:,3],
             'Rrup':     r_rup,
             'Y':        deltaT,
            }
fname_stan_data = dir_out + fname_out + '_stan_data' + '.json'

#create output directory
pathlib.Path(dir_out).mkdir(parents=True, exist_ok=True) 

#write as json file
cmdstanpy.utils.write_stan_json(fname_stan_data, stan_data)

# run stan
# ---   ---   ---   ---
#compile stan model
stan_model = cmdstanpy.CmdStanModel(stan_file=fname_stan_model) 
stan_model.compile(force=True)
#run full MCMC sampler
stan_fit = stan_model.sample(data=fname_stan_data, chains=n_chains, 
                             iter_warmup=n_iter_warmup, iter_sampling=n_iter_sampling,
                             seed=1, refresh=10, max_treedepth=max_treedepth, adapt_delta=adapt_delta,
                             show_progress=True, show_console=True, output_dir=dir_out+'stan_fit/' )

#delete json files
fname_dir = np.array( os.listdir(dir_out) )
#velocity filenames
fname_json = fname_dir[ [bool(re.search('\.json$',f_d)) for f_d in fname_dir] ]
for f_j in fname_json: os.remove(dir_out + f_j)

### Postprocessing
#### Extract posterior samples

In [ ]:
#sample names
col_names_dB  = ['deltaB.%i'%(k)   for k in range(n_eq)]
col_names_dBP = ['deltaBP.%i'%(k)  for k in range(n_eq)]
col_names_dS  = ['deltaS.%i'%(k)   for k in range(n_sta)]
col_names_all = col_names_dB + col_names_dBP + col_names_dS

#extract sample
stan_posterior = stan_fit.stan_variable('deltaB')
stan_posterior = np.concatenate((stan_posterior, stan_fit.stan_variable('deltaBP')), axis=1)
stan_posterior = np.concatenate((stan_posterior, stan_fit.stan_variable('deltaS')), axis=1)

#save raw-posterior distribution
df_stan_posterior_raw = pd.DataFrame(stan_posterior, columns = col_names_all)
df_stan_posterior_raw.to_csv(dir_out + fname_out + '_stan_posterior_raw' + '.csv', index=False)

del col_names_dB, col_names_dBP, col_names_dS
del stan_posterior

##### Summarize Random Effects

In [ ]:
#ground motion information
df_gminfo = df_totres[['motion_id', 'event_id', 'event_name', 'station_id', 'path_id',
                       'mag', 'mag2', 'ztor', 'vs30',  'gs', 'rrup']]
df_gminfo.loc[:,['motion_id','event_id','station_id']] = df_gminfo[['motion_id','event_id','station_id']].astype(int)

#summarize residuals
#between-event
delta_dB_med  = np.array([ np.median(   df_stan_posterior_raw.loc[:,f'deltaB.{k}'] )       for k in range(n_eq)])[eq_inv]
delta_dB_mu   = np.array([ np.mean(     df_stan_posterior_raw.loc[:,f'deltaB.{k}'] )       for k in range(n_eq)])[eq_inv]
delta_dB_sig  = np.array([ np.std(      df_stan_posterior_raw.loc[:,f'deltaB.{k}'] )       for k in range(n_eq)])[eq_inv]
delta_dB_p16  = np.array([ np.quantile( df_stan_posterior_raw.loc[:,f'deltaB.{k}'], 0.16)  for k in range(n_eq)])[eq_inv]
delta_dB_p84  = np.array([ np.quantile( df_stan_posterior_raw.loc[:,f'deltaB.{k}'], 0.84)  for k in range(n_eq)])[eq_inv]
#between-event-path
delta_dBP_med = np.array([ np.median(   df_stan_posterior_raw.loc[:,f'deltaBP.{k}'] )      for k in range(n_eq)])[eq_inv]
delta_dBP_mu  = np.array([ np.mean(     df_stan_posterior_raw.loc[:,f'deltaBP.{k}'] )      for k in range(n_eq)])[eq_inv]
delta_dBP_sig = np.array([ np.std(      df_stan_posterior_raw.loc[:,f'deltaBP.{k}'] )      for k in range(n_eq)])[eq_inv]
delta_dBP_p16 = np.array([ np.quantile( df_stan_posterior_raw.loc[:,f'deltaBP.{k}'], 0.16) for k in range(n_eq)])[eq_inv]
delta_dBP_p84 = np.array([ np.quantile( df_stan_posterior_raw.loc[:,f'deltaBP.{k}'], 0.84) for k in range(n_eq)])[eq_inv]
#between-site
delta_dS_med  = np.array([ np.median(   df_stan_posterior_raw.loc[:,f'deltaS.{k}'] )       for k in range(n_sta)])[sta_inv]
delta_dS_mu   = np.array([ np.mean(     df_stan_posterior_raw.loc[:,f'deltaS.{k}'] )       for k in range(n_sta)])[sta_inv]
delta_dS_sig  = np.array([ np.std(      df_stan_posterior_raw.loc[:,f'deltaS.{k}'] )       for k in range(n_sta)])[sta_inv]
delta_dS_p16  = np.array([ np.quantile( df_stan_posterior_raw.loc[:,f'deltaS.{k}'], 0.16)  for k in range(n_sta)])[sta_inv]
delta_dS_p84  = np.array([ np.quantile( df_stan_posterior_raw.loc[:,f'deltaS.{k}'], 0.84)  for k in range(n_sta)])[sta_inv]
#within-event
delta_dWS_med = deltaT - (delta_dB_med + delta_dBP_med + delta_dS_med)
delta_dWS_mu  = deltaT - (delta_dB_mu  + delta_dBP_mu  + delta_dS_mu)

#summarize random effects
rndeff_summary = np.vstack((delta_dB_mu,
                            delta_dB_med, 
                            delta_dB_sig,
                            delta_dB_p16,
                            delta_dB_p84,
                            delta_dBP_mu,
                            delta_dBP_med, 
                            delta_dBP_sig,
                            delta_dBP_p16,
                            delta_dBP_p84,
                            delta_dS_mu,
                            delta_dS_med, 
                            delta_dS_sig,
                            delta_dS_p16,
                            delta_dS_p84,
                            delta_dWS_med,
                            delta_dWS_mu)).T
columns_names = ['dB_mean',  'dB_med',  'dB_std',  'dB_prc0.16',  'dB_prc0.84',
                 'dBP_mean', 'dBP_med', 'dBP_std', 'dBP_prc0.16', 'dBP_prc0.84',
                 'dS_mean',  'dS_med',  'dS_std',  'dS_prc0.16',  'dS_prc0.84',
                 'dWS_mean', 'dWS_med']
df_rndeff_summary = pd.DataFrame(rndeff_summary, columns = columns_names, index=df_gminfo.index)
#create dataframe with random effect summary
df_rndeff_summary = pd.merge(df_gminfo, df_rndeff_summary, how='right', left_index=True, right_index=True)
df_rndeff_summary.to_csv(dir_out + fname_out + '_stan_parameters' + '.csv', index=False)

In [ ]:
# #create and save trace plots
# dir_fig_summ = dir_out  + 'summary_figs/'
# #create figures directory if doesn't exit
# pathlib.Path(dir_fig_summ).mkdir(parents=True, exist_ok=True) 

# #create stan trace plots
# #stan_az_fit = az.from_cmdstanpy(stan_fit)
# stan_az_fit = az.InferenceData(posterior=stan_fit.draws_xr())
# for j in np.random.randint(low=0, high=len(col_names_all), size=(20,)):
#     c_name = col_names_all[j]
#     #create trace plot with arviz
#     ax = az.plot_trace(stan_az_fit,  var_names=c_name, figsize=(10,5) ).ravel()
#     ax[0].yaxis.set_major_locator(plt_autotick())
#     ax[0].set_xlabel('sample value')
#     ax[0].set_ylabel('frequency')
#     ax[0].set_title('')
#     ax[0].grid(axis='both')
#     ax[1].set_xlabel('iteration')
#     ax[1].set_ylabel('sample value')
#     ax[1].grid(axis='both')
#     ax[1].set_title('')
#     fig = ax[0].figure
#     fig.suptitle(c_name)
#     fig.savefig(dir_fig_summ + fname_out_main + '_stan_traceplot_' + c_name + '_arviz' + '.png')

### Comparison

In [ ]:
#scatter plot deltaB
fname_fig = (fname_out + '_deltaB_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-3,3], [-3,3], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[eq_idx,'dB'], df_rndeff_summary.loc[eq_idx,'dB_mean'], 's', markersize=6, label='Mean')
hl2 = ax.plot(df_totres.loc[eq_idx,'dB'], df_rndeff_summary.loc[eq_idx,'dB_med'], 'o', markersize=4, label='Median')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta B$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#scatter plot deltaBP
fname_fig = (fname_out + '_deltaBP_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-0.01,0.01], [-0.01,0.01], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[eq_idx,'dBP'], df_rndeff_summary.loc[eq_idx,'dBP_mean'], 's', markersize=6, label='Mean')
hl2 = ax.plot(df_totres.loc[eq_idx,'dBP'], df_rndeff_summary.loc[eq_idx,'dBP_med'],  'o', markersize=4, label='Median')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta BP$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#scatter plot deltaS
fname_fig = (fname_out + '_deltaS_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-3,3], [-3,3], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[sta_idx,'dS'], df_rndeff_summary.loc[sta_idx,'dS_mean'], 's', markersize=6, label='Mean')
hl2 = ax.plot(df_totres.loc[sta_idx,'dS'], df_rndeff_summary.loc[sta_idx,'dS_med'],  'o', markersize=4, label='Median')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta S$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#scatter plot deltaWS
fname_fig = (fname_out + '_deltaWS_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-3,3], [-3,3], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[:,'dWS'], df_rndeff_summary.loc[:,'dWS_mean'], 's', markersize=6, label='Mean')
hl2 = ax.plot(df_totres.loc[:,'dWS'], df_rndeff_summary.loc[:,'dWS_med'],  'o', markersize=4, label='Median')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta WS$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

### Compare Error Prediction

In [ ]:
#sensitivity plot deltaB
fname_fig = (fname_out + '_deltaB_diff_sens').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([1,max(eq_cnt)], [0,0], linewidth=2, color='k')
hl1 = ax.plot(eq_cnt, df_rndeff_summary.loc[eq_idx,'dB_mean']-df_totres.loc[eq_idx,'dB'], 'o', markersize=6, label='Mean')
hl2 = ax.errorbar(eq_cnt, df_rndeff_summary.loc[eq_idx,'dB_mean']-df_totres.loc[eq_idx,'dB'], 
                  yerr=df_rndeff_summary.loc[eq_idx,'dB_std'],
                  capsize=4, fmt='none', ecolor=hl1[0].get_color(), label='16/84 Percentile')
#edit properties
ax.set_xlabel(r'Number of records', fontsize=30)
ax.set_ylabel(r'Difference (Estimated-Prescribed)',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta B$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#sensitivity plot deltaBP
fname_fig = (fname_out + '_deltaBP_diff_sens').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([1,max(eq_cnt)], [0,0], linewidth=2, color='k')
hl1 = ax.plot(eq_cnt, df_rndeff_summary.loc[eq_idx,'dBP_mean']-df_totres.loc[eq_idx,'dBP'], 'o', markersize=6, label='Mean')
hl2 = ax.errorbar(eq_cnt, df_rndeff_summary.loc[eq_idx,'dBP_mean']-df_totres.loc[eq_idx,'dBP'], 
                  yerr=df_rndeff_summary.loc[eq_idx,'dBP_std'],
                  capsize=4, fmt='none', ecolor=hl1[0].get_color(), label='16/84 Percentile')
#edit properties
ax.set_xlabel(r'Number of records', fontsize=30)
ax.set_ylabel(r'Difference (Estimated-Prescribed)',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta BP$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#sensitivity plot deltaS
fname_fig = (fname_out + '_deltaS_diff_sens').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
#hl0 = ax.plot([1,max(sta_cnt)], [0,0], linewidth=2, color='k')
hl0 = ax.plot([1,max(sta_cnt)], [0,0], linewidth=2, color='k')
hl1 = ax.plot(sta_cnt, df_rndeff_summary.loc[sta_idx,'dS_mean']-df_totres.loc[sta_idx,'dS'], 'o', markersize=6, label='Mean')
hl2 = ax.errorbar(sta_cnt, df_rndeff_summary.loc[sta_idx,'dS_mean']-df_totres.loc[sta_idx,'dS'], 
                  yerr=df_rndeff_summary.loc[sta_idx,'dS_std'],
                  capsize=4, fmt='none', ecolor=hl1[0].get_color(), label='16/84 Percentile')
#edit properties
ax.set_xlabel(r'Number of records', fontsize=30)
ax.set_ylabel(r'Difference (Estimated-Prescribed)',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta S$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

### Compare Standard Deviations

In [ ]:
#extracted random terms
deltaB  = df_rndeff_summary.loc[eq_idx,'dB_mean'].values
deltaBP = df_rndeff_summary.loc[eq_idx,'dBP_mean'].values
deltaS  = df_rndeff_summary.loc[sta_idx,'dS_mean'].values

#count bins
cnt4cmp = [[1,2],[2,10],[5,10],[10,25],[25,50],[50,100],[40,10000]]
cnt4cmp_lgd = ['%i-%i'%(c4cmp[0], c4cmp[1]) for c4cmp in cnt4cmp ] 

#deltaB
deltaB_binned_std = [ np.std( deltaB[np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1])] )  for c4cmp in cnt4cmp ]
deltaB_binned_cnt = [ np.sum( np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1]) )          for c4cmp in cnt4cmp ]
#deltaBP
deltaS_binned_std = [ np.std( deltaBP[np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1])] ) for c4cmp in cnt4cmp ]
deltaS_binned_cnt = [ np.sum( np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1]) )          for c4cmp in cnt4cmp ]
#deltaS
deltaS_binned_std = [ np.std( deltaS[np.logical_and(c4cmp[0]<=sta_cnt, sta_cnt<c4cmp[1])] )  for c4cmp in cnt4cmp ]
deltaS_binned_cnt = [ np.sum( np.logical_and(c4cmp[0]<=sta_cnt, sta_cnt<c4cmp[1]) )          for c4cmp in cnt4cmp ]

In [ ]:
#extracted random terms
deltaB  = df_rndeff_summary.loc[eq_idx,'dB_mean'].values  / stan_data['sclB']
deltaBP = df_rndeff_summary.loc[eq_idx,'dBP_mean'].values / stan_data['sclBP']
deltaS  = df_rndeff_summary.loc[sta_idx,'dS_mean'].values / stan_data['sclS']

#count bins
cnt4cmp = [[1,2],[2,5],[5,15],[15,25],[25,50],[50,100],[100,10000]]
cnt4cmp_lgd = ['%i-%i'%(c4cmp[0], c4cmp[1]) for c4cmp in cnt4cmp ] 

#deltaB
deltaB_binned_std  = [ np.std( deltaB[np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1])] )  for c4cmp in cnt4cmp ]
deltaB_binned_cnt  = [ np.sum( np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1]) )          for c4cmp in cnt4cmp ]
#deltaBP
deltaBP_binned_std = [ np.std( deltaBP[np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1])] ) for c4cmp in cnt4cmp ]
deltaBP_binned_cnt = [ np.sum( np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1]) )          for c4cmp in cnt4cmp ]
#deltaS
deltaS_binned_std  = [ np.std( deltaS[np.logical_and(c4cmp[0]<=sta_cnt, sta_cnt<c4cmp[1])] )  for c4cmp in cnt4cmp ]
deltaS_binned_cnt  = [ np.sum( np.logical_and(c4cmp[0]<=sta_cnt, sta_cnt<c4cmp[1]) )          for c4cmp in cnt4cmp ]

#deltaB standard deviation comparison
fname_fig = (fname_out + '_deltaB_std').replace(' ','_')
fig, ax = plt.subplots(figsize = (17,10), nrows=2)
#compare standard deviation
hl0 = ax[0].plot( [-1,len(cnt4cmp)],  np.full(2,tau0),        linewidth=2, color='k', label=r'$\tau_0$')
hl1 = ax[0].plot( range(len(cnt4cmp)), deltaB_binned_std, 'o', markersize=6)
#edit properties
ax[0].grid(which='both')
ax[0].set_xlim([-1,len(cnt4cmp)])
ax[0].set_ylim([0,1.0])
ax[0].set_xticks(range(len(cnt4cmp)))
ax[0].set_xticklabels([])
ax[0].tick_params(axis='x', labelsize=25)
ax[0].tick_params(axis='y', labelsize=25)
ax[0].legend(loc='lower right', fontsize=30)
ax[0].set_ylabel(f'Empirical\nStandard Deviation',  fontsize=30)
ax[0].set_title(r'Comparison Std $\delta B$', fontsize=30)
#number of data points
hl0 = ax[1].bar(range(len(cnt4cmp)),  deltaB_binned_cnt)
#edit properties
ax[1].grid(which='both')
ax[1].set_xlim([-1,len(cnt4cmp)])
ax[1].set_xticks(range(len(cnt4cmp)))
ax[1].set_xticklabels(cnt4cmp_lgd)
ax[1].set_xlabel(r'Bin size (Number of Events)', fontsize=30)
ax[1].set_ylabel(r'Sample size',  fontsize=30)
ax[1].tick_params(axis='x', labelsize=25)
ax[1].tick_params(axis='y', labelsize=25)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#deltaBP standard deviation comparison
fname_fig = (fname_out + '_deltaBP_std').replace(' ','_')
fig, ax = plt.subplots(figsize = (17,10), nrows=2)
#compare standard deviation
hl0 = ax[0].plot( [-1,len(cnt4cmp)],  np.full(2,tauP),         linewidth=2, color='k', label=r'$\tau_P$')
hl1 = ax[0].plot( range(len(cnt4cmp)), deltaBP_binned_std, 'o', markersize=6)
#edit properties
ax[0].grid(which='both')
ax[0].set_xlim([-1,len(cnt4cmp)])
ax[0].set_ylim([0,0.004])
ax[0].set_xticks(range(len(cnt4cmp)))
ax[0].set_xticklabels([])
ax[0].tick_params(axis='x', labelsize=25)
ax[0].tick_params(axis='y', labelsize=25)
ax[0].legend(loc='lower right', fontsize=30)
ax[0].set_ylabel(f'Empirical\nStandard Deviation',  fontsize=30)
ax[0].set_title(r'Comparison Std $\delta BP$', fontsize=30)
#number of data points
hl0 = ax[1].bar(range(len(cnt4cmp)),  deltaB_binned_cnt)
#edit properties
ax[1].grid(which='both')
ax[1].set_xlim([-1,len(cnt4cmp)])
ax[1].set_xticks(range(len(cnt4cmp)))
ax[1].set_xticklabels(cnt4cmp_lgd)
ax[1].set_xlabel(r'Bin size (Number of Events)', fontsize=30)
ax[1].set_ylabel(r'Sample size',  fontsize=30)
ax[1].tick_params(axis='x', labelsize=25)
ax[1].tick_params(axis='y', labelsize=25)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#deltaS standard deviation comparison
fname_fig = (fname_out + '_deltaS_std').replace(' ','_')
fig, ax = plt.subplots(figsize = (17,10), nrows=2)
#compare standard deviation
hl0 = ax[0].plot( [-1,len(cnt4cmp)],  np.full(2,phiS),        linewidth=2, color='k', label='$\phi_S$')
hl1 = ax[0].plot( range(len(cnt4cmp)), deltaS_binned_std, 'o', markersize=6)
#edit properties
ax[0].grid(which='both')
ax[0].set_xlim([-1,len(cnt4cmp)])
ax[0].set_ylim([0,0.6])
ax[0].set_xticks(range(len(cnt4cmp)))
ax[0].set_xticklabels([])
ax[0].tick_params(axis='x', labelsize=25)
ax[0].tick_params(axis='y', labelsize=25)
ax[0].legend(loc='lower right', fontsize=30)
ax[0].set_ylabel(f'Empirical\nStandard Deviation',  fontsize=30)
ax[0].set_title(r'Comparison Std $\delta S$', fontsize=30)
#number of data points
hl0 = ax[1].bar(range(len(cnt4cmp)),  deltaS_binned_cnt)
#edit properties
ax[1].grid(which='both')
ax[1].set_xlim([-1,len(cnt4cmp)])
ax[1].set_xticks(range(len(cnt4cmp)))
ax[1].set_xticklabels(cnt4cmp_lgd)
ax[1].set_xlabel(r'Bin size (Number of Stations)', fontsize=30)
ax[1].set_ylabel(r'Sample size',  fontsize=30)
ax[1].tick_params(axis='x', labelsize=25)
ax[1].tick_params(axis='y', labelsize=25)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

In [ ]:
#initalize adjusted random terms
deltaB_adj  = np.zeros(deltaB.size)
deltaBP_adj = np.zeros(deltaBP.size)
deltaS_adj  = np.zeros(deltaS.size)

#adjust random terms for std
for j, c4cmp in enumerate(cnt4cmp):
    i_eq  = np.logical_and(c4cmp[0]<=eq_cnt, eq_cnt<c4cmp[1])
    i_sta = np.logical_and(c4cmp[0]<=sta_cnt, sta_cnt<c4cmp[1]) 
    #
    deltaB_adj[i_eq]  = deltaB[i_eq]  * tau0 / deltaB_binned_std[j]
    deltaBP_adj[i_eq] = deltaBP[i_eq] * tauP / deltaBP_binned_std[j]
    deltaS_adj[i_sta] = deltaS[i_sta] * phiS / deltaS_binned_std[j]


#
df_rndeff_summary_adj = df_rndeff_summary.copy()
df_rndeff_summary_adj.loc[:,'dB_adj']  = (deltaB_adj  * stan_data['sclB'])[eq_inv]
df_rndeff_summary_adj.loc[:,'dBP_adj'] = (deltaBP_adj * stan_data['sclBP'])[eq_inv]
df_rndeff_summary_adj.loc[:,'dS_adj']  = (deltaS_adj  * stan_data['sclS'])[sta_inv]
#
df_rndeff_summary_adj.loc[:,'dWS_adj'] = stan_data['Y'] - df_rndeff_summary_adj[['dB_adj','dBP_adj','dS_adj']].sum(axis=1)

In [ ]:


#scatter plot adjusted deltaB
fname_fig = (fname_out + '_deltaB_adj_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-3,3], [-3,3], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[eq_idx,'dB'], df_rndeff_summary_adj.loc[eq_idx,'dB_mean'], 's', markersize=6, label='Original')
hl2 = ax.plot(df_totres.loc[eq_idx,'dB'], df_rndeff_summary_adj.loc[eq_idx,'dB_adj'],  'o', markersize=4, label='Adjusted')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta B$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#scatter plot adjusted deltaBP
fname_fig = (fname_out + '_deltaBP_adj_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-0.01,0.01], [-0.01,0.01], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[eq_idx,'dBP'], df_rndeff_summary_adj.loc[eq_idx,'dBP_mean'], 's', markersize=6, label='Original')
hl2 = ax.plot(df_totres.loc[eq_idx,'dBP'], df_rndeff_summary_adj.loc[eq_idx,'dBP_adj'],  'o', markersize=4, label='Adjusted')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta BP$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#scatter plot adjusted deltaS
fname_fig = (fname_out + '_deltaS_adj_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-3,3], [-3,3], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[sta_idx,'dS'], df_rndeff_summary_adj.loc[sta_idx,'dS_mean'], 's', markersize=6, label='Original')
hl2 = ax.plot(df_totres.loc[sta_idx,'dS'], df_rndeff_summary_adj.loc[sta_idx,'dS_adj'],  'o', markersize=4, label='Adjusted')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta S$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )

#scatter plot deltaWS
fname_fig = (fname_out + '_deltaWS_adj_scatter').replace(' ','_')
fig, ax = plt.subplots(figsize = (10,10))
hl0 = ax.plot([-3,3], [-3,3], linewidth=2, color='k')
hl1 = ax.plot(df_totres.loc[:,'dWS'], df_rndeff_summary_adj.loc[:,'dWS_mean'], 's', markersize=6, label='Original')
hl2 = ax.plot(df_totres.loc[:,'dWS'], df_rndeff_summary_adj.loc[:,'dWS_adj'],  'o', markersize=4, label='Adjusted')
#edit properties
ax.set_xlabel(r'Prescribed', fontsize=30)
ax.set_ylabel(r'Estimated',  fontsize=30)
ax.legend(loc='lower right', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
ax.set_title(r'Comparison $\delta WS$', fontsize=30)
fig.tight_layout()
fig.savefig( dir_fig + fname_fig + '.png' )